In [1]:
import numpy as np
import pandas as pd

In [2]:
class multiclass():
    def __init__(self,input = 3, capas=1,nodos=1,clases = 3):
        self.capas = capas
        self.nodos = nodos
        self.forma = (self.capas,self.nodos)
        self.clases = clases
        self.input = input
    def inicializar(self):
        self.red = []
        w = np.random.normal(size = (self.input,self.nodos))
        b = np.random.normal(size = self.nodos)
        self.red.append([w,b])
        for capa in range(self.capas):
            w = np.random.normal(size = (self.nodos,self.nodos))
            b = np.random.normal(size = self.nodos)
            self.red.append([w,b])
        w_f = np.random.normal(size = [self.nodos,self.clases])
        b_f = np.random.normal(size = self.clases)
        self.red.append([w_f,b_f])
    def mutar(self,alfa=0.1):
        red = []
        for r in self.red:
            r0 = r[0]+alfa*np.random.normal(size = r[0].shape)
            r1 = r[1]+alfa*np.random.normal(size = r[1].shape)
            red.append([r0,r1])
        hijo = multiclass(self.input,self.capas,self.nodos,self.clases)
        hijo.inicializar()
        hijo.red = red
        return hijo
    def cruzar(self,other):
        if self.forma == other.forma:
            hijo = multiclass(self.input,self.capas,self.nodos,self.clases)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (self.red[i][0] + other.red[i][0])/2.0
                r1 = (self.red[i][1] + other.red[i][1])/2.0
                red.append([r0,r1])
            hijo.red = red
            return hijo
        else:
            return None
    def sigmoid(self,x):
        return 1.0/(1.0 + np.exp(-x))
    def softmax(self,x):
        x = np.exp(x)/(np.exp(x).sum())
        return x
    def predecir(self,entrada):
        for rr in range(len(self.red)):
            r = self.red[rr]
            entrada = np.dot(entrada,r[0])
            entrada = entrada + r[1]
            entrada = self.sigmoid(entrada)
        entrada = self.softmax(entrada)
        entrada = entrada.argmax()
        return entrada
    def evaluar(self,entrada,resultado):
        entrada = self.predecir(entrada)
        if float(entrada) == float(resultado):
            return 1
        return 0

        
            
    

In [3]:
def crear_poblacion(poblacion_por_especie,especies,input,nodos,clases):
    poblacion = []
    for i in range(poblacion_por_especie):
        for j in especies:
            individuo = multiclass(input = input,capas = j,nodos = nodos,clases = clases)
            individuo.inicializar()
            poblacion.append(individuo)
    return poblacion

In [4]:
def evaluar_poblacion(poblacion,muestra):
    scores = []
    for p in poblacion:
        score = 0
        count = 0
        for r in muestra.iterrows():
            v = r[1].target
            u = r[1]
            del u['target']
            score += p.evaluar(u.values,v)
            count += 1
        score = float(score / float(count))
        scores.append(score)
    return scores

In [5]:
def predecir(best_model,test):
    scores = []
    for r in test.iterrows():
        u = r[1]
        score = p.predecir(u)
        scores.append(score)
    return scores

In [6]:
titanic = pd.read_csv('mnist/train.csv')
titanic = titanic.rename(columns={'label':'target'})


In [7]:
titanic.target.head()

0    1
1    0
2    1
3    4
4    0
Name: target, dtype: int64

In [8]:
titanic_train = titanic.sample(38000)
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]



In [9]:
titanic_train.target.head()

11054    8
28458    8
33353    5
5448     2
6473     1
Name: target, dtype: int64

In [10]:
poblacion = crear_poblacion(1000,[1],784,64,10)
poblacion = pd.DataFrame(list(zip(poblacion,range(len(poblacion)))))
poblacion.columns = ['model','score']

In [ ]:

for i in range(1000):
    alfa = np.random.random()
    scores = evaluar_poblacion(poblacion.model.tolist(),titanic_train.sample(200))
    poblacion.score = scores
    print(i,poblacion.score.mean(),poblacion.score.max(),alfa)
    poblacion = poblacion.sort_values('score',ascending=False)[0:int(len(poblacion)/2)]
    nueva_poblacion = []
    alfa = 1
    for p in poblacion.model.tolist():
        hijo = p.mutar(alfa = alfa)
        nueva_poblacion.append(hijo)
    nueva_poblacion = pd.DataFrame(list(zip(nueva_poblacion,range(len(nueva_poblacion)))))
    nueva_poblacion.columns = ['model','score']
    poblacion = poblacion.append(nueva_poblacion).reset_index(drop=True)


/home/jesus/anaconda3/envs/tensorflow/lib/python3.4/site-packages/ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in exp


0 0.10087 0.2 0.3105140892870344
1 0.10655 0.22 0.5798592040049259
2 0.10818000000000001 0.22 0.998728894777393
3 0.10820500000000001 0.195 0.2955461310958072
4 0.10939000000000002 0.2 0.7343114298602379
5 0.11248 0.22 0.7737437429235202
6 0.11585000000000001 0.21 0.3182467141572356
7 0.12461000000000001 0.235 0.18968937719803025
8 0.12406500000000001 0.215 0.8675807440933987
9 0.12392 0.24 0.2924781102522853
10 0.12147500000000001 0.215 0.7032767974240229
11 0.128505 0.245 0.15740771691785882
12 0.12066500000000001 0.23 0.9388790029269494
13 0.125545 0.235 0.07540776903776236
14 0.122295 0.22 0.9379042388022037
15 0.125235 0.22 0.5682145536005321
16 0.13177 0.225 0.11599411867931797
17 0.13587 0.245 0.930193990330323
18 0.14539999999999997 0.235 0.2765831930251911
19 0.14417000000000002 0.24 0.04027348727507751
20 0.13167500000000001 0.225 0.8614046818894187
21 0.161 0.26 0.7736452045586976
22 0.14977499999999996 0.265 0.7530527216933524
23 0.13980499999999998 0.245 0.8856892283230295

In [ ]:
scores = evaluar_poblacion(poblacion.model.tolist(),titanic_train)
poblacion.score = scores
poblacion.sort_values('score',ascending=False)

In [ ]:
best_model = poblacion.sort_values('score',ascending = False).iloc[0,0]

In [ ]:
titanic = pd.read_csv('mnist/test.csv')


In [ ]:
titanic.head()

In [ ]:
resultado = pd.read_csv('mnist/test.csv')
scores = predecir(best_model,titanic)
resultado['Label'] = scores
resultado.Label = resultado.Label.round().astype(int)
resultado['ImageId']=list(range(1,len(resultado)+1))

resultado[['ImageId','Label']].to_csv('mnist/result.csv',index=False)